In [24]:
from easyocr import Reader
import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import re

In [3]:
# PARAMTERI
languages_list = ['en', 'pt']
print(languages_list)

gpu = True

['en', 'pt']


In [50]:
img = cv2.imread("../images/file_3.jpg")
cv2.imshow("Slika", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
original = img.copy()

In [52]:
reader = Reader(languages_list, gpu)
results = reader.readtext(img)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [53]:
results

[([[np.int32(475), np.int32(649)],
   [np.int32(517), np.int32(649)],
   [np.int32(517), np.int32(685)],
   [np.int32(475), np.int32(685)]],
  '€:',
  np.float64(0.03976555031945574)),
 ([[np.int32(638), np.int32(664)],
   [np.int32(676), np.int32(664)],
   [np.int32(676), np.int32(696)],
   [np.int32(638), np.int32(696)]],
  '7',
  np.float64(0.08208541318746043)),
 ([[np.int32(378), np.int32(720)],
   [np.int32(410), np.int32(720)],
   [np.int32(410), np.int32(764)],
   [np.int32(378), np.int32(764)]],
  '0',
  np.float64(0.9979170689815788)),
 ([[np.int32(430), np.int32(714)],
   [np.int32(652), np.int32(714)],
   [np.int32(652), np.int32(788)],
   [np.int32(430), np.int32(788)]],
  '2 4 5 5',
  np.float64(0.8028466168725696)),
 ([[np.int32(709), np.int32(745)],
   [np.int32(785), np.int32(745)],
   [np.int32(785), np.int32(781)],
   [np.int32(709), np.int32(781)]],
  '^ <',
  np.float64(0.041750882254236656)),
 ([[np.int32(179), np.int32(791)],
   [np.int32(292), np.int32(791)],
  

In [54]:
def box_coordinates(box):
  (lt, rt, br, bl) = box
  lt = (int(lt[0]), int(lt[1]))
  rt = (int(rt[0]), int(rt[1]))
  br = (int(br[0]), int(br[1]))
  bl = (int(bl[0]), int(bl[1]))
  return lt, rt, br, bl

In [55]:
def write_text(text, x, y, img, color=(50,50,255), font_size=30):
  img_pil = Image.fromarray(img)
  draw = ImageDraw.Draw(img_pil)
  draw.text((x, y - font_size), text, fill = color)
  img = np.array(img_pil)
  return img

In [56]:
def draw_img(img, lt, br, color=(200,255,0),thickness=2):
  cv2.rectangle(img, lt, br, color, thickness)
  return img

In [57]:
img = original.copy()
for (box, text, probability) in results:
  print(box, text, probability)
  lt, rt, br, bl = box_coordinates(box)
  img = draw_img(img, lt, br)
  img = write_text(text, lt[0], lt[1], img)
cv2.imshow("Slika", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[np.int32(475), np.int32(649)], [np.int32(517), np.int32(649)], [np.int32(517), np.int32(685)], [np.int32(475), np.int32(685)]] €: 0.03976555031945574
[[np.int32(638), np.int32(664)], [np.int32(676), np.int32(664)], [np.int32(676), np.int32(696)], [np.int32(638), np.int32(696)]] 7 0.08208541318746043
[[np.int32(378), np.int32(720)], [np.int32(410), np.int32(720)], [np.int32(410), np.int32(764)], [np.int32(378), np.int32(764)]] 0 0.9979170689815788
[[np.int32(430), np.int32(714)], [np.int32(652), np.int32(714)], [np.int32(652), np.int32(788)], [np.int32(430), np.int32(788)]] 2 4 5 5 0.8028466168725696
[[np.int32(709), np.int32(745)], [np.int32(785), np.int32(745)], [np.int32(785), np.int32(781)], [np.int32(709), np.int32(781)]] ^ < 0.041750882254236656
[[np.int32(179), np.int32(791)], [np.int32(292), np.int32(791)], [np.int32(292), np.int32(836)], [np.int32(179), np.int32(836)]] 35ase 0.029072877507564456
[[np.int32(331), np.int32(804)], [np.int32(443), np.int32(804)], [np.int32(443), 

In [62]:
extracted_text = ' '.join([text[1] for text in results])
print("Extracted Text:", extracted_text)
type(extracted_text)

Extracted Text: €: 7 0 2 4 5 5 ^ < 35ase 2 Yire 70/e014 2804 19- 2 5v -01-4 NO: o Uncr 2 7wef { 2 # 3-Scon } b 78 222: 8223 


str

In [65]:
# Use regex to find all digits in the extracted text
pattern = r'\b\d\b'
meter_values = "".join(re.findall(pattern, extracted_text))
# Print the found digits
print("Extracted Digits:", meter_values)

Extracted Digits: 702455224223


In [46]:
import torch
print(torch.cuda.is_available())

False
